In [10]:
import warnings
warnings.filterwarnings('ignore')
import sklearn
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.models import Sequential

In [2]:
X,y=make_moons(n_samples=500,noise=0.30,random_state=42)

In [9]:
# X

In [5]:
y

array([1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0,
       0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1,

In [6]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42)

**Voting_classifier with logistic regression,Random forest,SVC**

In [7]:
log_clf = LogisticRegression(solver="lbfgs", random_state=42)
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)
svm_clf = SVC(gamma="scale", random_state=42,probability=True)

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard')

In [8]:
voting_clf.fit(X_train,y_train)

VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=42,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                               

In [11]:
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.864
RandomForestClassifier 0.896
SVC 0.896
VotingClassifier 0.912


**Voting_classifer with logistic regression,Random Forest,SVC and deep learning **

In [15]:
# model 1
def build_nn():
    model= Sequential([Dense(50,activation='relu',input_shape=[2]),
                        #Dense(25,activation='relu'),
                        Dense(1,activation='sigmoid')
                       ])

    model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [13]:
m2 = tf.keras.wrappers.scikit_learn.KerasClassifier(build_nn,epochs=500,verbose=False)

In [14]:
m2._estimator_type = "classifier"

In [16]:
# Model2
def build_nn2():
    model2= Sequential([Dense(50,activation='relu',input_shape=[2]),
                      Dense(25,activation='relu'),
                      #Dropout(0.4),
                      Dense(25,activation='relu'),
                      Dense(1,activation='sigmoid')
                           ])
    model2.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model2



In [17]:
m3 = tf.keras.wrappers.scikit_learn.KerasClassifier(build_nn2,epochs=500,verbose=False)

In [18]:
m3._estimator_type = "classifier"

In [22]:
voting2 = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf),('Dense',m2),('Dense2',m3)],
    voting='soft',flatten_transform=True)

In [23]:
for clf in (log_clf, rnd_clf, svm_clf,m2,m3,voting2):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))
    print('-'*80)

LogisticRegression 0.864
--------------------------------------------------------------------------------
RandomForestClassifier 0.896
--------------------------------------------------------------------------------
SVC 0.896
--------------------------------------------------------------------------------
Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
KerasClassifier 0.872
--------------------------------------------------------------------------------
KerasClassifier 0.896
--------------------------------------------------------------------------------
Instructions for updating:
Please use `model.predict()` instead.
VotingClassifier 0.904
-----------------------------------------------------------------